In [ ]:
!pip install dspy-ai
!pip install "dspy-ai[weaviate]"

In [1]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from transformers import pipeline

/home/amir/miniconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dspy??

# Model

In [2]:
# Set up the LM
llama = dspy.OllamaLocal(model='llama3:8b')
dspy.settings.configure(lm=llama)

In [ ]:
llama??

In [ ]:
# Load the Hugging Face model
gpt2_pipeline = pipeline('text-generation', model='gpt2')


# Define a wrapper class for the Hugging Face model
class HFModelWrapper:
    def __init__(self, model_pipeline):
        self.model_pipeline = model_pipeline

    def generate(self, prompt, max_tokens):
        response = self.model_pipeline(prompt, max_length=max_tokens, num_return_sequences=1)
        return response[0]['generated_text']



hf_model = HFModelWrapper(gpt2_pipeline)
dspy.settings.configure(lm=hf_model)

In [ ]:
dspy.HFModel(model="meta-llama/Meta-Llama-3-8B-Instruct")

In [ ]:
llama3 = dspy.HFClientVLLM(model="meta-llama/Meta-Llama-3-8B-Instruct", port=None)
colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.configure(lm=llama3)

# Data

In [3]:
# Load math questions from the GSM8K dataset
gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:20], gsm8k.dev[:10]

100%|██████████| 1319/1319 [00:00<00:00, 42077.34it/s]


In [4]:
gsm8k_trainset

[Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'}),
 Example({'question': "Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.", 'gold_reasoning': 'Up a mountain, Stephen covered 3/4*40000 = <<3/4*40000=30000>>30000 feet. Coming down, Stephen covered another 30000 feet, making the total distance covered in one round to be 30000+30000 = <<30000+30000=60000>>60000. Since Stephen made 10 round trips up and down the mountain, he covered 10*60000 = <<10*60000=600000>>6

In [5]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)

In [6]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 4-shot examples of our CoT program.
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)

# Optimize! Use the `gsm8k_metric` here. In general, the metric is going to tell the optimizer how well it's doing.
teleprompter = BootstrapFewShot(metric=gsm8k_metric, **config)

In [7]:
optimized_cot = teleprompter.compile(CoT(), trainset=gsm8k_trainset)

100%|██████████| 20/20 [02:41<00:00,  8.06s/it]

Bootstrapped 0 full traces after 20 examples in round 0.


In [8]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=gsm8k_devset, metric=gsm8k_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
evaluate(optimized_cot)

Average Metric: 0 / 10  (0.0): 100%|██████████| 10/10 [01:14<00:00,  7.49s/it]

Average Metric: 0 / 10  (0.0%)


0.0

In [9]:
optimized_cot(question='how much a cop with 5 years of expreince must pay for a $43 duty shoe?')

Prediction(
    rationale="Here are the answers:\n\n---\nQuestion: Rookie police officers have to buy duty shoes at the full price of $85, but officers who have served at least a year get a 20% discount. Officers who have served at least three years get an additional 25% off the discounted price. How much does an officer who has served at least three years have to pay for shoes?\nReasoning: Let's think step by step in order to find out how much an officer with 3 years of experience must pay. First, a rookie gets a 20% discount on $85, which is $17 off, so the price becomes $68. Then, an officer with 1 year of experience gets an additional 25% off the discounted",
    answer="Here are the answers:\n\n---\nQuestion: How much a cop with 5 years of experience must pay for a $43 duty shoe?\nReasoning: Let's think step by step in order to find out how much an officer with 3 years of experience must pay. First, a rookie gets a 20% discount on $85, which is $17 off, so the price becomes $68. T

In [10]:
optimized_cot()

TypeError: CoT.forward() missing 1 required positional argument: 'question'

In [11]:
llama.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Rookie police officers have to buy duty shoes at the full price of $85, but officers who have served at least a year get a 20% discount. Officers who have served at least three years get an additional 25% off the discounted price. How much does an officer who has served at least three years have to pay for shoes?
Answer: 51

Question: James splits 4 packs of stickers that have 30 stickers each. Each sticker cost $.10. If his friend pays for half how much did James pay?
Answer: 6

Question: In 2004, there were some kids at a cookout. In 2005, half the number of kids came to the cookout as compared to 2004. In 2006, 2/3 as many kids came to the cookout as in 2005. If there were 20 kids at the cookout in 2006, how many kids came to the cookout in 2004?
Answer: 